<h1 style="color:#6495ED;">SQLAlchemy</h1>

<p>SQLAlchemy é uma biblioteca de mapeamento objeto-relacional para Python. <br />
Neste é possível utilizar bancos de dados com Python, tanto através da definição de classes quanto através de código SQL.</p>


<p><span style="color: crimson;">Observação:</span> Nos exemplos deste notebook será utilizado o SQLite e a tabela abaixo descrita. Para visualização do banco de dados será utilizado o SQLiteStudio.</p>

<pre>
CREATE TABLE carro(
    cd_carro INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_marca VARCHAR(50),
    nr_placa VARCHAR(10),
    nm_cor VARCHAR(10)
);
</pre>

<p>Antes de iniciar, é necessário instalar o SQLAlchemy:</p>

In [ ]:
%pip install sqlalchemy

<h3 style="color:#6495ED;">Criação da engine:</h3>

<p>Para conectar a um banco de dados suportado pelo SQLAlchemy é necessário instanciar um objeto da classe <span style="color:#3DCAB5;">Engine</span> utilizando a função <span style="color:khaki;">create_engine</span><br />
A string de conexão que é passada como parâmetro para o <span style="color:khaki;">create_engine</span> muda de acordo com o banco de dados alvo.</p>

In [9]:
from sqlalchemy import create_engine

DATABASE_NAME = "aula.db"

engine = create_engine(f"sqlite:///{DATABASE_NAME}")
engine

Engine(sqlite:///aula.db)

<h3 style="color:#6495ED;">Criação da sessão:</h3>

<p>Quando criada uma engine, ela pode utilizada para a criação de uma <span style="color:#3DCAB5;">Sessão</span> utilizando a função <span style="color:khaki;">sessionmaker</span>.<br />
Comando SQL devem ser executados a partir de uma sessão.</p>

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session

<h3 style="color:#6495ED;">Executando comandos SQL:</h3>

<p>Utiliza-se o método <span style="color:khaki;">execute</span> para executar comandos SQL no banco de dados.<br />
Sessões funcionam no formato transaction, de forma que é necessário <span style="color:khaki;">commit</span> para executar comandos DML no BD.</p>

In [ ]:
session.execute("""CREATE TABLE carro(
    cd_carro INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_marca VARCHAR(50),
    nr_placa VARCHAR(10),
    nm_cor VARCHAR(10)
    );""")

In [ ]:
session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Fiat', 'ABC-1234', 'Prata')""")
session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Volvo', 'ABC-9876', 'Branco')""")
session.commit()

<p>Caso haja um erro, é possível retornar a sessão ao estado anterior, não propagando as modificações, utilizando o <span style="color:khaki;">rollback</span>:</p>

In [ ]:
retorno = session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Ford', 'ABC-4567', 'Preto')""")
session.rollback()

Consulta:

In [ ]:
results = session.execute("""SELECT * FROM carro""")
results

In [ ]:
for result in results:
    print(result)

In [ ]:
class Carro:

    def __init__(self, cd_carro, nm_marca, nr_placa, nm_cor) -> None:
        self.cd_carro = cd_carro
        self.nm_marca = nm_marca
        self.nr_placa = nr_placa
        self.nm_cor = nm_cor

In [ ]:
results = session.execute("""SELECT * FROM carro""")
results

In [ ]:
for result in results:
    carro = Carro(result[0], result[1], result[2], result[3])
    print(carro.__dict__)

In [ ]:
results = session.execute("""SELECT * FROM carro WHERE cd_carro = 1 LIMIT 1""")
results.__next__()

<p><b>Uma sessão deve sempre ser encerrada, utilizando o <span style="color:khaki;">close</span>:</b></p>

In [ ]:
session.close()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
engine = create_engine("""sqlite:///aula.db""")
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.execute("""CREATE TABLE cliente(
    cd_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_cliente VARCHAR(50) NOT NULL,
    nr_cpf VARCHAR(11) NOT NULL,
    dsc_endereco VARCHAR(70),
    renda NUMBER
    )""")

In [ ]:
session.execute("""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
    VALUES ('Fulano', '10', 'Rua Bla 10', 1200)""")
session.execute("""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
    VALUES ('Beltrano', '11', 'Rua Ble 100', 1400)""")
session.commit()
session.close()

In [1]:
class Cliente:
    
    def __init__(self, nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente=None) -> None:
        self.cd_cliente = cd_cliente
        self.nm_cliente = nm_cliente
        self.nr_cpf = nr_cpf
        self.dsc_endereco = dsc_endereco
        self.renda = renda

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from typing import List, Union

class ClienteDAO:

    def __init__(self) -> None:
        DATABASE_NAME = "aula.db"
        Session = sessionmaker(bind=create_engine(f"sqlite:///{DATABASE_NAME}"))
        self.session = Session()
    
    def __del__(self) -> None:
        self.session.close()

    def save(self, cliente: Cliente) -> None:
        self.session.execute(f"""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
        VALUES ('{cliente.nm_cliente}', '{cliente.nr_cpf}', '{cliente.dsc_endereco}', '{cliente.renda}')""")
        self.session.commit()

    def query(self, cd_cliente: int) -> Union[Cliente,None]:
        results = self.session.execute(f"""SELECT nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente FROM cliente
                                           WHERE cd_cliente = {cd_cliente} LIMIT 1""")
        try:
            result = results.__next__()
            cliente = Cliente(result[0], result[1], result[2], result[3], result[4])
            return cliente
        except StopIteration:
            return None

    def query_all(self) -> List[Cliente]:
        resultados = []
        results = self.session.execute(f"""SELECT nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente FROM cliente""")
        for result in results:
            cliente = Cliente(result[0], result[1], result[2], result[3], result[4])
            resultados.append(cliente)
        return resultados


In [3]:
cliente = Cliente('Cicrano', '20', 'Rua Sei lá 201', 1500)
dao = ClienteDAO()
dao.save(cliente)

In [5]:
cliente = dao.query(1)
cliente.__dict__

{'cd_cliente': None,
 'nm_cliente': 'Fulano',
 'nr_cpf': '10',
 'dsc_endereco': 'Rua Bla 10',
 'renda': 1200}

In [7]:
clientes = dao.query_all()
for cliente in clientes:
    print(cliente.__dict__)

{'cd_cliente': None, 'nm_cliente': 'Fulano', 'nr_cpf': '10', 'dsc_endereco': 'Rua Bla 10', 'renda': 1200}
{'cd_cliente': None, 'nm_cliente': 'Beltrano', 'nr_cpf': '11', 'dsc_endereco': 'Rua Ble 100', 'renda': 1400}
{'cd_cliente': None, 'nm_cliente': 'Cicrano', 'nr_cpf': '20', 'dsc_endereco': 'Rua Sei lá 201', 'renda': 1500}


<h3 style="color:#6495ED;">ORM:</h3>

<p>Além de poder se executar comandos SQL o SQLAlchemy também possuí um pacote de ORM ( Object-relational mapping ) que permite mapear as tabelas do banco em classes e objetos. Para isso, utilizamos a função <span style="color:khaki";>declarative_base</span> do pacote <span style="color:#3DCAB5;">sqlalchemy.orm</span></p>

In [1]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [2]:
from sqlalchemy import Column, Integer, String

class CarroTable(Base):
    __tablename__ = "carro"

    cd_carro = Column("cd_carro", Integer, primary_key=True)
    nm_marca = Column("nm_marca", String(50))
    nr_placa = Column("nr_placa", String(10))
    nm_cor = Column("nm_cor", String(10))

In [3]:
carro = CarroTable()
carro.nm_marca = "Ford"
carro.nr_placa = "ABC-1010"
carro.nm_cor = "Azul"

INSERT:

In [4]:
Session = sessionmaker(bind=engine)
session = Session()
session.add(carro)
session.commit()
session.close()

NameError: name 'sessionmaker' is not defined

SELECT:

In [16]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).all()

for result in results:
    print(result.__dict__)

session.close()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D91F1A88>, 'nm_cor': 'Prata', 'cd_carro': 1, 'nm_marca': 'Fiat', 'nr_placa': 'ABC-1234'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D91F1148>, 'nm_cor': 'Branco', 'cd_carro': 2, 'nm_marca': 'Volvo', 'nr_placa': 'ABC-9876'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D91F11C8>, 'nm_cor': 'Branco', 'cd_carro': 3, 'nm_marca': 'Volvo', 'nr_placa': 'ABC-9876'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D91F1808>, 'nm_cor': 'Azul', 'cd_carro': 4, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D91F12C8>, 'nm_cor': 'Azul', 'cd_carro': 5, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}


SELECT BY ID:

In [17]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter_by(cd_carro=1)

for result in results:
    print(result.__dict__)

session.close()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D90BC248>, 'nm_cor': 'Prata', 'cd_carro': 1, 'nm_marca': 'Fiat', 'nr_placa': 'ABC-1234'}


In [18]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter_by(nm_cor='Azul')

for result in results:
    print(result.__dict__)

session.close()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D8050CC8>, 'nm_cor': 'Azul', 'cd_carro': 4, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D922B088>, 'nm_cor': 'Azul', 'cd_carro': 5, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}


In [19]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter(CarroTable.nm_cor =='Azul')

for result in results:
    print(result.__dict__)

session.close()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D922B888>, 'nm_cor': 'Azul', 'cd_carro': 4, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D922B908>, 'nm_cor': 'Azul', 'cd_carro': 5, 'nm_marca': 'Ford', 'nr_placa': 'ABC-1010'}


EXERCICIO:

In [20]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [21]:
from sqlalchemy import Column, Integer, String, Float

class Cliente(Base):
    __tablename__ = "cliente"

    cd_cliente = Column("cd_cliente", Integer, primary_key=True)
    nm_cliente = Column("nm_cliente", String(50))
    nr_cpf = Column("nr_cpf", String(11))
    dsc_endereco = Column("dsc_endereco", String(70))
    renda = Column("renda", Float)

In [22]:
Session = sessionmaker(bind=engine)
session = Session()

In [23]:
cliente = Cliente()
cliente.nm_cliente = "Batman"
cliente.nr_cpf = "100"
cliente.dsc_endereco = "BatCaverna"
cliente.renda = 1

In [24]:
session.add(cliente)
session.commit()

In [25]:
results = session.query(Cliente).all()
for result in results:
    print(result.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D9221C88>, 'nm_cliente': 'Fulano', 'dsc_endereco': 'Rua Bla 10', 'cd_cliente': 1, 'nr_cpf': '10', 'renda': 1200}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D9221C08>, 'nm_cliente': 'Beltrano', 'dsc_endereco': 'Rua Ble 100', 'cd_cliente': 2, 'nr_cpf': '11', 'renda': 1400}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D9221D88>, 'nm_cliente': 'Cicrano', 'dsc_endereco': 'Rua Sei lá 201', 'cd_cliente': 3, 'nr_cpf': '20', 'renda': 1500}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D9221348>, 'nm_cliente': 'Batman', 'dsc_endereco': 'BatCaverna', 'cd_cliente': 4, 'nr_cpf': '100', 'renda': 1}


In [26]:
results = session.query(Cliente).filter_by(cd_cliente=4)
for result in results:
    print(result.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D9221348>, 'nm_cliente': 'Batman', 'dsc_endereco': 'BatCaverna', 'cd_cliente': 4, 'nr_cpf': '100', 'renda': 1}


In [27]:
results = session.query(Cliente).filter(Cliente.renda > 1200, Cliente.renda <= 1500)
for result in results:
    print(result.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D90B6B08>, 'nm_cliente': 'Beltrano', 'dsc_endereco': 'Rua Ble 100', 'cd_cliente': 2, 'nr_cpf': '11', 'renda': 1400}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D1D90B6B48>, 'nm_cliente': 'Cicrano', 'dsc_endereco': 'Rua Sei lá 201', 'cd_cliente': 3, 'nr_cpf': '20', 'renda': 1500}


In [ ]:
session.close()

<h3 style="color:#6495ED;">Dataclasses:</h3>

In [ ]:
from dataclasses import dataclass

In [ ]:
from typing import Optional

@dataclass
class Carro:
    cd_carro: int
    nm_marca: str
    nr_placa: str
    nm_cor: Optional[str] = None

In [ ]:
carro = Carro(1, "Citroen", "ABC-2020", "Vermelho")

In [ ]:
print(carro)

<h3 style="color:#6495ED;">Agrupando as duas estruturas:</h3>

In [ ]:
class CarroTable(Base):
    __tablename__ = "carro"

    cd_carro = Column("cd_carro", Integer, primary_key=True)
    nm_marca = Column("nm_marca", String(50))
    nr_placa = Column("nr_placa", String(10))
    nm_cor = Column("nm_cor", String(10))

In [ ]:
@dataclass
class Carro(CarroTable):
    cd_carro: int
    nm_marca: str
    nr_placa: str
    nm_cor: Optional[str] = None

In [ ]:
carro = Carro(4, "Citroen", nr_placa="ABC-2020", nm_cor="Prata")